Instalación de la librería "procyclingstats" e importación del resto de librerías necesarias para la realización del proyecto 

In [ ]:
pip install procyclingstats

In [2]:
from procyclingstats import RaceClimbs
from procyclingstats import Race
from procyclingstats import Stage
from procyclingstats import RaceStartlist
import requests
from bs4 import BeautifulSoup
import pandas as pd
import dtale
# from pprint import pprint
# from lxml import html
# from lxml.html import fromstring
# import urllib.request
# from urllib.request import urlopen
# import random
# import re
# import scrapy

Accedemos al database de dicha página, y más concretamente, al apartado de "startlist" para recabar la lista de participantes en el Tour de Francia 2023

In [3]:
race_startlist = RaceStartlist("race/tour-de-france/2023/startlist")
startlist = race_startlist.startlist()

len(startlist)



176

In [4]:
# Crear un dataframe con los datos de tour de france

startlist_df = pd.DataFrame(startlist)

startlist_df

,rider_name,rider_url,nationality,rider_number,team_name,team_url
0,VINGEGAARD Jonas,rider/jonas-vingegaard-rasmussen,DK,1,Jumbo-Visma (WT),team/team-jumbo-visma-2023
1,BENOOT Tiesj,rider/tiesj-benoot,BE,2,Jumbo-Visma (WT),team/team-jumbo-visma-2023
2,KELDERMAN Wilco,rider/wilco-kelderman,NL,3,Jumbo-Visma (WT),team/team-jumbo-visma-2023
3,KUSS Sepp,rider/sepp-kuss,US,4,Jumbo-Visma (WT),team/team-jumbo-visma-2023
4,LAPORTE Christophe,rider/christophe-laporte,FR,5,Jumbo-Visma (WT),team/team-jumbo-visma-2023
...,...,...,...,...,...,...
171,CRAS Steff,rider/steff-cras,BE,214,TotalEnergies (PRT),team/team-totalenergies-2023
172,FERRON Valentin,rider/valentin-ferron,FR,215,TotalEnergies (PRT),team/team-totalenergies-2023
173,LATOUR Pierre,rider/pierre-latour,FR,216,TotalEnergies (PRT),team/team-totalenergies-2023
174,OSS Daniel,rider/daniel-oss,IT,217,TotalEnergies (PRT),team/team-totalenergies-2023


Procederemos a anlizar el dataframe e iniciar su limpieza

In [5]:
# Ver la información del dataframe

startlist_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   rider_name    176 non-null    object
 1   rider_url     176 non-null    object
 2   nationality   176 non-null    object
 3   rider_number  176 non-null    int64 
 4   team_name     176 non-null    object
 5   team_url      176 non-null    object
dtypes: int64(1), object(5)
memory usage: 8.4+ KB


In [6]:
# Comprobar si hay duplicados

startlist_df.duplicated().sum()

0

In [7]:
# Eliminar las columnas "rider_url","team_url" y "rider_number"

startlist_df = startlist_df.drop(["rider_url","team_url","rider_number"], axis=1)

startlist_df

,rider_name,nationality,team_name
0,VINGEGAARD Jonas,DK,Jumbo-Visma (WT)
1,BENOOT Tiesj,BE,Jumbo-Visma (WT)
2,KELDERMAN Wilco,NL,Jumbo-Visma (WT)
3,KUSS Sepp,US,Jumbo-Visma (WT)
4,LAPORTE Christophe,FR,Jumbo-Visma (WT)
...,...,...,...
171,CRAS Steff,BE,TotalEnergies (PRT)
172,FERRON Valentin,FR,TotalEnergies (PRT)
173,LATOUR Pierre,FR,TotalEnergies (PRT)
174,OSS Daniel,IT,TotalEnergies (PRT)


In [8]:
# Crear una columna llamada rider_name_id que se obtenga de la columna "rider_name" y que contenga el id del ciclista

startlist_df["rider_name_id"] = startlist_df["rider_name"].str.split("/").str[2]

startlist_df

,rider_name,nationality,team_name,rider_name_id
0,VINGEGAARD Jonas,DK,Jumbo-Visma (WT),NaN
1,BENOOT Tiesj,BE,Jumbo-Visma (WT),NaN
2,KELDERMAN Wilco,NL,Jumbo-Visma (WT),NaN
3,KUSS Sepp,US,Jumbo-Visma (WT),NaN
4,LAPORTE Christophe,FR,Jumbo-Visma (WT),NaN
...,...,...,...,...
171,CRAS Steff,BE,TotalEnergies (PRT),NaN
172,FERRON Valentin,FR,TotalEnergies (PRT),NaN
173,LATOUR Pierre,FR,TotalEnergies (PRT),NaN
174,OSS Daniel,IT,TotalEnergies (PRT),NaN


In [9]:
# Llenar la columna "rider_name_id" con numeros del 1 al 176

startlist_df["rider_name_id"] = range(1, len(startlist_df) + 1)

startlist_df

,rider_name,nationality,team_name,rider_name_id
0,VINGEGAARD Jonas,DK,Jumbo-Visma (WT),1
1,BENOOT Tiesj,BE,Jumbo-Visma (WT),2
2,KELDERMAN Wilco,NL,Jumbo-Visma (WT),3
3,KUSS Sepp,US,Jumbo-Visma (WT),4
4,LAPORTE Christophe,FR,Jumbo-Visma (WT),5
...,...,...,...,...
171,CRAS Steff,BE,TotalEnergies (PRT),172
172,FERRON Valentin,FR,TotalEnergies (PRT),173
173,LATOUR Pierre,FR,TotalEnergies (PRT),174
174,OSS Daniel,IT,TotalEnergies (PRT),175


In [10]:
# Poner la columna "rider_name_id" al principio del dataframe

startlist_df = startlist_df[["rider_name_id","rider_name","nationality","team_name"]]

startlist_df

,rider_name_id,rider_name,nationality,team_name
0,1,VINGEGAARD Jonas,DK,Jumbo-Visma (WT)
1,2,BENOOT Tiesj,BE,Jumbo-Visma (WT)
2,3,KELDERMAN Wilco,NL,Jumbo-Visma (WT)
3,4,KUSS Sepp,US,Jumbo-Visma (WT)
4,5,LAPORTE Christophe,FR,Jumbo-Visma (WT)
...,...,...,...,...
171,172,CRAS Steff,BE,TotalEnergies (PRT)
172,173,FERRON Valentin,FR,TotalEnergies (PRT)
173,174,LATOUR Pierre,FR,TotalEnergies (PRT)
174,175,OSS Daniel,IT,TotalEnergies (PRT)


In [ ]:
# cambiar los elementos de la columna "rider_name_id a minusculas y cambiar los espacios por guiones bajos

startlist_df['rider_name'] = startlist_df['rider_name'].str.lower().str.replace(' ', '_')

startlist_df

In [ ]:
# separar los elementos entre parentesis de la columna "team_name" a una nueva columna llamada "ranking"

startlist_df['ranking'] = startlist_df['team_name'].str.split("(").str[1]

startlist_df

In [ ]:
# eliminar los elementos entre parentesis de la columna "team_name"

startlist_df['team_name'] = startlist_df['team_name'].str.split("(").str[0]

startlist_df

In [ ]:
# quitar los parentesis de la columna "ranking"

startlist_df['ranking'] = startlist_df['ranking'].str.split(")").str[0]

startlist_df

In [ ]:
# cambiar a minusculas los elementos de la columna "ranking"

startlist_df['ranking'] = startlist_df['ranking'].str.lower()

startlist_df

In [ ]:
# cambiar los elementos de la columna "team_name" a minusculas

startlist_df['team_name'] = startlist_df['team_name'].str.lower()

startlist_df

# eliminar los espaciones y los guiones de "team_name" y cambarlos por guiones bajos

startlist_df['team_name'] = startlist_df['team_name'].str.replace(' ', '_').str.replace('-', '_')

startlist_df

# eliminar los guiones bajos del final de los elementos de la columna "team_name"

startlist_df['team_name'] = startlist_df['team_name'].str.rstrip('_')

startlist_df

# cambiar los dobres guiones bajos por un solo guion bajo

startlist_df['team_name'] = startlist_df['team_name'].str.replace('__', '_')

startlist_df

# mostrar todo el dataframe

pd.set_option('display.max_rows', None)

startlist_df

In [ ]:
# cambiar los elementos de la columna "nationality" a minusculas

startlist_df["nationality"] = startlist_df['nationality'].str.lower()

startlist_df

In [157]:
# Exportar a csv los datos de tour de france

startlist_df.to_csv('startlist.csv', index=False)

Para la extración de los puertos de las 21 etapas crearemos una función que vaya iterando por la url. Retornará las cimas con su correspondiente ID de cada etapa

In [5]:
def getStages(maxEtapa):
    stage_climbs=[]
    for stages in range(1,maxEtapa+1):    
        stage = Stage("race/tour-de-france/2023/stage-{}".format(stages))
        climbs = stage.climbs()
        for x in climbs:
            x["stage_id"] = stages
        stage_climbs.append(climbs)

    # Convertir la lista anidada en una lista plana de diccionarios
    flat_data = [item for sublist in stage_climbs for item in sublist]

    # Crear un DataFrame a partir de la lista de diccionarios
    stages = pd.DataFrame(flat_data)

    # Mostrar el DataFrame
    return stages

maxEtapa = 21
stages = getStages(maxEtapa)
stages

# Exportar a csv los datos de los puertos de montaña de tour de france

stages.to_csv('climbs_clean.csv', index=False)


In [8]:
# importar el csv de climbs_clean.csv y climbs.csv

climbs_clean = pd.read_csv("climbs_clean.csv")

climbs_clean

climbs = pd.read_csv("C:/Users/luiso/IRONHACK/w4-project_ETL/climbs.csv")

climbs

,climb_name,climb_url,length,steepness,top,km_before_finnish
0,Col de la Loze,location/col-de-la-loze,28.3,6.0,2300,7
1,Cormet de Roselend,location/cormet-de-roselend,19.8,6.1,1971,99
2,Grand Colombier,location/col-du-grand-colombier,17.8,7.0,1494,0
3,Col du Tourmalet,location/col-du-tourmalet,17.0,7.4,2112,47
4,Cauterets-Cambrasque,location/cauterets-cambasque,16.2,5.3,1353,0
...,...,...,...,...,...,...
65,Côte de Chambéry-le-Haut,location/cote-de-chambery-le-haut,1.6,4.3,345,123
66,Côte de Masmont,location/cote-de-masmont,1.3,5.3,345,16
67,Côte du Pavé des Gardes,location/cote-du-pave-des-gardes,1.3,5.9,172,72
68,Côte de Condat-sur-Vienne,location/cote-de-condat-sur-vienne,1.2,5.3,282,10


Para fortalecer la tabla, haremos un merge del csv "climbs", que contiene todas las cimas ordenadas por sus características, al csv "climbs_clean. El resultado será un dataframe mucho más completo, y que limpiaremos para hacerlo más homogéneo

In [9]:
# hacer un merge de stage1_df con climb_df para tener un dataframe con los datos de stage1 y climb

stages_climbs_df = pd.merge(climbs_clean, climbs, on='climb_name')

stages_climbs_df

,climb_name,climb_url_x,stage_id,climb_url_y,length,steepness,top,km_before_finnish
0,Côte de Laukiz,location/cote-de-laukiz,1,location/cote-de-laukiz,2.1,6.8,220,168
1,Côte de San Juan de Gaztelugatxe,location/alto-de-san-juan-de-gaztelugatxe,1,location/alto-de-san-juan-de-gaztelugatxe,3.6,7.9,297,114
2,Col de Morga,location/alto-de-morga,1,location/alto-de-morga,3.8,4.9,324,41
3,Côte de Vivero,location/alto-el-vivero,1,location/alto-el-vivero,4.3,7.0,355,27
4,Côte de Pike,location/alto-pike,1,location/alto-pike,2.0,9.8,210,10
...,...,...,...,...,...,...,...,...
65,Col de Grosse Pierre,location/col-de-grosse-pierre,20,location/col-de-grosse-pierre,3.2,7.9,937,69
66,Col de la Schlucht,location/col-de-la-schlucht,20,location/col-de-la-schlucht,4.2,5.0,1136,54
67,Petit Ballon,location/petit-ballon,20,location/petit-ballon,9.3,8.1,1157,25
68,Col du Platzerwasel,location/col-du-platzerwasel,20,location/col-du-platzerwasel,7.1,8.3,1179,8


In [10]:
# crear una columna de climb_id al principio del dataframe

stages_climbs_df.insert(0, 'climb_id', range(1, 1 + len(stages_climbs_df)))

stages_climbs_df

,climb_id,climb_name,climb_url_x,stage_id,climb_url_y,length,steepness,top,km_before_finnish
0,1,Côte de Laukiz,location/cote-de-laukiz,1,location/cote-de-laukiz,2.1,6.8,220,168
1,2,Côte de San Juan de Gaztelugatxe,location/alto-de-san-juan-de-gaztelugatxe,1,location/alto-de-san-juan-de-gaztelugatxe,3.6,7.9,297,114
2,3,Col de Morga,location/alto-de-morga,1,location/alto-de-morga,3.8,4.9,324,41
3,4,Côte de Vivero,location/alto-el-vivero,1,location/alto-el-vivero,4.3,7.0,355,27
4,5,Côte de Pike,location/alto-pike,1,location/alto-pike,2.0,9.8,210,10
...,...,...,...,...,...,...,...,...,...
65,66,Col de Grosse Pierre,location/col-de-grosse-pierre,20,location/col-de-grosse-pierre,3.2,7.9,937,69
66,67,Col de la Schlucht,location/col-de-la-schlucht,20,location/col-de-la-schlucht,4.2,5.0,1136,54
67,68,Petit Ballon,location/petit-ballon,20,location/petit-ballon,9.3,8.1,1157,25
68,69,Col du Platzerwasel,location/col-du-platzerwasel,20,location/col-du-platzerwasel,7.1,8.3,1179,8


In [11]:
# Eliminar las columnas "climb_url_x" y "climb_url_y"

stages_climbs_df = stages_climbs_df.drop(["climb_url_x","climb_url_y"], axis=1)

stages_climbs_df

,climb_id,climb_name,stage_id,length,steepness,top,km_before_finnish
0,1,Côte de Laukiz,1,2.1,6.8,220,168
1,2,Côte de San Juan de Gaztelugatxe,1,3.6,7.9,297,114
2,3,Col de Morga,1,3.8,4.9,324,41
3,4,Côte de Vivero,1,4.3,7.0,355,27
4,5,Côte de Pike,1,2.0,9.8,210,10
...,...,...,...,...,...,...,...
65,66,Col de Grosse Pierre,20,3.2,7.9,937,69
66,67,Col de la Schlucht,20,4.2,5.0,1136,54
67,68,Petit Ballon,20,9.3,8.1,1157,25
68,69,Col du Platzerwasel,20,7.1,8.3,1179,8


In [12]:
# cambiar los elementos de la columna a minusculas y cambiar los espacios por guiones bajos

stages_climbs_df['climb_name'] = stages_climbs_df['climb_name'].str.lower().str.replace(' ', '_')

stages_climbs_df

,climb_id,climb_name,stage_id,length,steepness,top,km_before_finnish
0,1,côte_de_laukiz,1,2.1,6.8,220,168
1,2,côte_de_san_juan_de_gaztelugatxe,1,3.6,7.9,297,114
2,3,col_de_morga,1,3.8,4.9,324,41
3,4,côte_de_vivero,1,4.3,7.0,355,27
4,5,côte_de_pike,1,2.0,9.8,210,10
...,...,...,...,...,...,...,...
65,66,col_de_grosse_pierre,20,3.2,7.9,937,69
66,67,col_de_la_schlucht_,20,4.2,5.0,1136,54
67,68,petit_ballon,20,9.3,8.1,1157,25
68,69,col_du_platzerwasel,20,7.1,8.3,1179,8


In [13]:
# exportar a csv los datos de stages_climbs_df

stages_climbs_df.to_csv('stages_climbs_clean.csv', index=False)

In [273]:
# abrir el csv de stages

stages_df = pd.read_csv('stages.csv')

stages_df

,Unnamed: 0,stages,length,winner,leader,stage_id
0,0,Bilbao - Bilbao,182,"Yates,Adam","Yates,Adam",1
1,1,Vitoria-Gasteiz - Donostia,208,"Lafay,Victor","Yates,Adam",2
2,2,Amorebieta - Bayonne,193,"Philipsen,Jasper","Yates,Adam",3
3,3,Dax - Nogaro,181,"Philipsen,Jasper","Yates,Adam",4
4,4,Pau - Laruns,162,"Hindley,Jai","Hindley,Jai",5
5,5,Tarbes - Cauterets,144,"Pogačar,Tadej","Vingegaard,Jonas",6
6,6,Mont-de-Marsan - Bordeaux,169,"Philipsen,Jasper","Vingegaard,Jonas",7
7,7,Libourne - Limoges,200,"Pedersen,Mads","Vingegaard,Jonas",8
8,8,Saint-Léonard-de-Noblat - Puy de,182,"Woods,Michael","Vingegaard,Jonas",9
9,9,Vulcania - Issoire,167,"Bilbao,Pello","Vingegaard,Jonas",10


In [274]:
# eliminar la columna unnamed:0
    
stages_df

,Unnamed: 0,stages,length,winner,leader,stage_id
0,0,Bilbao - Bilbao,182,"Yates,Adam","Yates,Adam",1
1,1,Vitoria-Gasteiz - Donostia,208,"Lafay,Victor","Yates,Adam",2
2,2,Amorebieta - Bayonne,193,"Philipsen,Jasper","Yates,Adam",3
3,3,Dax - Nogaro,181,"Philipsen,Jasper","Yates,Adam",4
4,4,Pau - Laruns,162,"Hindley,Jai","Hindley,Jai",5
5,5,Tarbes - Cauterets,144,"Pogačar,Tadej","Vingegaard,Jonas",6
6,6,Mont-de-Marsan - Bordeaux,169,"Philipsen,Jasper","Vingegaard,Jonas",7
7,7,Libourne - Limoges,200,"Pedersen,Mads","Vingegaard,Jonas",8
8,8,Saint-Léonard-de-Noblat - Puy de,182,"Woods,Michael","Vingegaard,Jonas",9
9,9,Vulcania - Issoire,167,"Bilbao,Pello","Vingegaard,Jonas",10


In [275]:
# eliminar la columna "unnamed:0"

stages_df1 = stages_df.drop(["Unnamed: 0"], axis=1)

stages_df

,Unnamed: 0,stages,length,winner,leader,stage_id
0,0,Bilbao - Bilbao,182,"Yates,Adam","Yates,Adam",1
1,1,Vitoria-Gasteiz - Donostia,208,"Lafay,Victor","Yates,Adam",2
2,2,Amorebieta - Bayonne,193,"Philipsen,Jasper","Yates,Adam",3
3,3,Dax - Nogaro,181,"Philipsen,Jasper","Yates,Adam",4
4,4,Pau - Laruns,162,"Hindley,Jai","Hindley,Jai",5
5,5,Tarbes - Cauterets,144,"Pogačar,Tadej","Vingegaard,Jonas",6
6,6,Mont-de-Marsan - Bordeaux,169,"Philipsen,Jasper","Vingegaard,Jonas",7
7,7,Libourne - Limoges,200,"Pedersen,Mads","Vingegaard,Jonas",8
8,8,Saint-Léonard-de-Noblat - Puy de,182,"Woods,Michael","Vingegaard,Jonas",9
9,9,Vulcania - Issoire,167,"Bilbao,Pello","Vingegaard,Jonas",10


In [276]:
# cambiar a minusculas los elementos de la columna "stages"

stages_df['stages'] = stages_df['stages'].str.lower()

stages_df

,Unnamed: 0,stages,length,winner,leader,stage_id
0,0,bilbao - bilbao,182,"Yates,Adam","Yates,Adam",1
1,1,vitoria-gasteiz - donostia,208,"Lafay,Victor","Yates,Adam",2
2,2,amorebieta - bayonne,193,"Philipsen,Jasper","Yates,Adam",3
3,3,dax - nogaro,181,"Philipsen,Jasper","Yates,Adam",4
4,4,pau - laruns,162,"Hindley,Jai","Hindley,Jai",5
5,5,tarbes - cauterets,144,"Pogačar,Tadej","Vingegaard,Jonas",6
6,6,mont-de-marsan - bordeaux,169,"Philipsen,Jasper","Vingegaard,Jonas",7
7,7,libourne - limoges,200,"Pedersen,Mads","Vingegaard,Jonas",8
8,8,saint-léonard-de-noblat - puy de,182,"Woods,Michael","Vingegaard,Jonas",9
9,9,vulcania - issoire,167,"Bilbao,Pello","Vingegaard,Jonas",10


In [278]:
# cambiar los elementos de la columna "winner" a minusculas y cambiar las comas por guiones bajos

stages_df['winner'] = stages_df['winner'].str.lower().str.replace(',', '_')

stages_df


,Unnamed: 0,stages,length,winner,leader,stage_id
0,0,bilbao - bilbao,182,yates_adam,"Yates,Adam",1
1,1,vitoria-gasteiz - donostia,208,lafay_victor,"Yates,Adam",2
2,2,amorebieta - bayonne,193,philipsen_jasper,"Yates,Adam",3
3,3,dax - nogaro,181,philipsen_jasper,"Yates,Adam",4
4,4,pau - laruns,162,hindley_jai,"Hindley,Jai",5
5,5,tarbes - cauterets,144,pogačar_tadej,"Vingegaard,Jonas",6
6,6,mont-de-marsan - bordeaux,169,philipsen_jasper,"Vingegaard,Jonas",7
7,7,libourne - limoges,200,pedersen_mads,"Vingegaard,Jonas",8
8,8,saint-léonard-de-noblat - puy de,182,woods_michael,"Vingegaard,Jonas",9
9,9,vulcania - issoire,167,bilbao_pello,"Vingegaard,Jonas",10


In [279]:
# cambiar los elementos de la columna "leader" a minusculas y cambiar las comas por guiones bajos

stages_df['leader'] = stages_df['leader'].str.lower().str.replace(',', '_')

stages_df

,Unnamed: 0,stages,length,winner,leader,stage_id
0,0,bilbao - bilbao,182,yates_adam,yates_adam,1
1,1,vitoria-gasteiz - donostia,208,lafay_victor,yates_adam,2
2,2,amorebieta - bayonne,193,philipsen_jasper,yates_adam,3
3,3,dax - nogaro,181,philipsen_jasper,yates_adam,4
4,4,pau - laruns,162,hindley_jai,hindley_jai,5
5,5,tarbes - cauterets,144,pogačar_tadej,vingegaard_jonas,6
6,6,mont-de-marsan - bordeaux,169,philipsen_jasper,vingegaard_jonas,7
7,7,libourne - limoges,200,pedersen_mads,vingegaard_jonas,8
8,8,saint-léonard-de-noblat - puy de,182,woods_michael,vingegaard_jonas,9
9,9,vulcania - issoire,167,bilbao_pello,vingegaard_jonas,10


In [280]:
# exportar a csv los datos de stages_df

stages_df.to_csv('stages.csv', index=False)



In [281]:
# abrir el csv de stages

stages_df = pd.read_csv('stages.csv')

stages_df

,stage,length,winner,leader,stage_id
0,bilbao - bilbao,182,yates_adam,yates_adam,1
1,vitoria-gasteiz - donostia,208,lafay_victor,yates_adam,2
2,amorebieta - bayonne,193,philipsen_jasper,yates_adam,3
3,dax - nogaro,181,philipsen_jasper,yates_adam,4
4,pau - laruns,162,hindley_jai,hindley_jai,5
5,tarbes - cauterets,144,pogačar_tadej,vingegaard_jonas,6
6,mont-de-marsan - bordeaux,169,philipsen_jasper,vingegaard_jonas,7
7,libourne - limoges,200,pedersen_mads,vingegaard_jonas,8
8,saint-léonard-de-noblat - puy de,182,woods_michael,vingegaard_jonas,9
9,vulcania - issoire,167,bilbao_pello,vingegaard_jonas,10


In [283]:
# cambiar la columna "stage_id" al principio del dataframe

stages_df1 = stages_df[["stage_id","stage","length","winner","leader"]]

stages_df1

,stage_id,stage,length,winner,leader
0,1,bilbao - bilbao,182,yates_adam,yates_adam
1,2,vitoria-gasteiz - donostia,208,lafay_victor,yates_adam
2,3,amorebieta - bayonne,193,philipsen_jasper,yates_adam
3,4,dax - nogaro,181,philipsen_jasper,yates_adam
4,5,pau - laruns,162,hindley_jai,hindley_jai
5,6,tarbes - cauterets,144,pogačar_tadej,vingegaard_jonas
6,7,mont-de-marsan - bordeaux,169,philipsen_jasper,vingegaard_jonas
7,8,libourne - limoges,200,pedersen_mads,vingegaard_jonas
8,9,saint-léonard-de-noblat - puy de,182,woods_michael,vingegaard_jonas
9,10,vulcania - issoire,167,bilbao_pello,vingegaard_jonas


In [284]:
# exportar a csv los datos de stages_df1

stages_df1.to_csv('stages.csv', index=False)

In [304]:
# importar el csv de withdrawals

withdrawals_df = pd.read_csv('withdrawals.csv')

withdrawals_df

,Unnamed: 0,withdrawal_id,stage_id,rider_name,team_name
0,0,1,1,mas_enric,movistar_team
1,1,2,2,carapaz_richard,ef_education_easypost
2,2,3,3,no_withdrawal,no_withdrawal
3,3,4,4,no_withdrawal,no_withdrawal
4,4,5,5,guarnieri_jacopo,lotto_dstny
5,5,6,5,sanchez_luis_leon,astana_qazaqstan_team
6,6,7,6,no_withdrawal,no_withdrawal
7,7,8,7,no_withdrawal,no_withdrawal
8,8,9,8,cavendish_mark,astana_qazaqstan_team
9,9,10,8,cras_steff,totalenergies


In [305]:
withdrawals_df = withdrawals_df.drop(["Unnamed: 0"], axis=1)

withdrawals_df

,withdrawal_id,stage_id,rider_name,team_name
0,1,1,mas_enric,movistar_team
1,2,2,carapaz_richard,ef_education_easypost
2,3,3,no_withdrawal,no_withdrawal
3,4,4,no_withdrawal,no_withdrawal
4,5,5,guarnieri_jacopo,lotto_dstny
5,6,5,sanchez_luis_leon,astana_qazaqstan_team
6,7,6,no_withdrawal,no_withdrawal
7,8,7,no_withdrawal,no_withdrawal
8,9,8,cavendish_mark,astana_qazaqstan_team
9,10,8,cras_steff,totalenergies


In [316]:
# exportar a csv los datos de withdrawals_df

withdrawals_df.to_csv('withdrawals.csv', index=False)

In [311]:
# hacer un merge de la columna "rider_name_id" de startlist_df con la columna "rider_name" de withdrawals_df

withdrawals_df2 = pd.merge(withdrawals_df, startlist_df, on='rider_name')

withdrawals_df2

,withdrawal_id,stage_id,rider_name,team_name_x,rider_name_id,nationality,team_name_y,ranking
0,1,1,mas_enric,movistar_team,105,es,movistar_team,wt
1,2,2,carapaz_richard,ef_education_easypost,33,ec,ef_education_easypost,wt
2,5,5,guarnieri_jacopo,lotto_dstny,150,it,lotto_dstny,prt
3,9,8,cavendish_mark,astana_qazaqstan_team,153,gb,astana_qazaqstan_team,wt
4,10,8,cras_steff,totalenergies,172,be,totalenergies,prt
5,11,9,simmons_quinn,lidl_trek,71,us,lidl_trek,wt
6,14,12,jakobsen_fabio,soudal_quick_step,46,nl,soudal_quick_step,wt
7,15,13,de_la_cruz_david,astana_qazaqstan_team,155,es,astana_qazaqstan_team,wt
8,16,13,turner_ben,ineos_grenadiers,24,gb,ineos_grenadiers,wt
9,17,13,ewan_caleb,lotto_dstny,145,au,lotto_dstny,prt


In [317]:
# exportar a csv los datos de withdrawals_df2

withdrawals_df2.to_csv('withdrawals2.csv', index=False)

In [322]:
# importar el csv de withdrawals2

withdrawals_df2 = pd.read_csv('withdrawals2.csv')

withdrawals_df2

# cambiar el nombre de la columna "team_name_y" a "team_name"

withdrawals_df2 = withdrawals_df2.rename(columns={'team_name_y': 'team_name'})

withdrawals_df2

# exportar a csv los datos de withdrawals_df2

withdrawals_df2.to_csv('withdrawals_clean.csv', index=False)

